### PDF Text Extraction

In [27]:
import pymupdf
import re

Examing a random page in the book to see the structure and quality of the extracted text.

In [28]:
doc = pymupdf.open("a guide to modern cookery.pdf")
out = open("output.txt", "wb")
i = 0
for page in doc:
    i += 1
    if i > 200:
        text = page.get_text()#.encode("utf8")
        # out.write(text)
    if i == 201:
        print(text)
    if i == 230:
        break 
out.close()

EGGS  175 
stove;  break  the  eggs  into  the  boiling  sauce,  season  with  a 
grain  of  salt,  and  put  the  cocottes,  one  by  one,  into  a  saute-pan 
containing  the  necessary  quantity  of  boiling  water. 
Poach  as  directed,  and  set  to  glaze  quickly  at  the  last 
moment. 
442— EQGS  EN  COCOTTE  WITH  CREAM 
This  preparation  constitutes  the  radical  type  of  this  series 
of  eggs,  and,  for  a  long  time,  was  the  only  one  in  use.  Heat 
the  cocottes  beforehand;  pour  a  tablespoonful  of  boiling  cream 
into  each,  followed  by  an  egg,  broken ;  season,  and  add  two 
little  lumps  of  butter,  the  size  of  peas.  Place  the  cocottes  in 
a  bain-marie,  and  poach  as  before. 
443— EaaS  EN  COCOTTE  A  LA  JEANNETTE 
Garnish  the  bottom  and  the  sides  of  the  cocottes  with  a 
thickness  of  one-third  inch  of  chicken-forcemeat  with  cream, 
mixed  with  a  fifth  of  its  volume  of  foie  gras.  Break  the  egg 
over  the 

Fantastic, the data seems to be well extracted and follow a consistent structure. Recipe titles are in all caps followed by instructions. There are some mistakes due to whatever OCR was originally used but they appear to be minor (e.g. "EQGS" instead of "EGGS".)

Now I'll open the PDF and examine it manually:
-pages 1-22 are the preface and table of contents.
- The glossary could be useful but is structured differently and not all entries are useful. I'll ignore it for now since most entries are elaborated on later.
- Each part appears to have a brief introduction before following the structure seen above. 
- Given that the page numbers of the book begin at PDF page 27, I'll extract each chapter separately to label their metadata according to the chapter title (e.g. "Stocks", "Sauces", "Soups", etc.) without having to iterate through the entire 1000 page PDF immediately.

Extracting the table of contents to get the titles and page numbers for each chapter.

In [29]:
doc = pymupdf.open("a guide to modern cookery.pdf")
toc_pages = [doc[20], doc[21]]
toc_page_texts = []

for page in toc_pages:
    text = page.get_text()
    toc_page_texts.append(text)
    print(text.encode("utf8"))

b"CONTENTS \nPART    I \nFUNDAMENTAL   ELEMENTS \nCHAPTER  I \nPAGE \nFONDS  DE  CUISINE  ........  I \nCHAPTER  II \nTHE  LEADING  WARM   SAUCES     .....  \xe2\x80\xa2  '5 \nCHAPTER  III \nTHE   SMALL  COMPOUND   SAUCES  ...  .  .  24 \nCHAPTER  IV \nCOLD  SAUCES  AND  COMPOUND  BUTTERS        .....  48 \nCHAPTER  V \nSAVOURY  JELLIES  OR  ASPICS  .  ......  59 \nCHAPTER  VI \nTHE  COURT-BOUILLONS  AND  THE  MARINADES  .  .  .  -64 \nCHAPTER  VII \n\\J/:  ELEMENTARY  PREPARATIONS  .....  70 \nCHAPTER  VIII \nTHE  VARIOUS  GARNISHES   FOR  SOUPS  .  .  .  .  87 \nCHAPTER  IX \nGARNISHING  PREPARATIONS   FOR  RELEVis  AND   ENTR]\xc2\xa3eS  .  .  92 \nCHAPTER  X \nU^DING  CULINARY  OPERATIONS  .  ....  97 \n"
b"xii  CONTENTS \nPART   II \nRECIPES  AND   MODES   OF  PROCEDURE \nCHAPTER  XI \nPAGE \nHORS-D'CEUVRES      .  .  .  .  .  .  .  ,  .137 \nCHAPTER  XII \nEGGS  .......  .  .        164 \nCHAPTER  XIII \nSOUPS  ..........      197 \nCHAPTER  XIV \nFISH  ..........        260 \nCH

In [30]:
chapters = {}

for text in toc_page_texts:
    lines = text.split("\n")
    for i, line in enumerate(lines):
        if i > 3:
            if i%2 == 1:
                # the title is the first words of the line 
                # followed by some amount of "."
                # the page number is at the end
                split_text = line.split(".")
                title = split_text[0]
                page_num = split_text[-1]
                chapters[title] = page_num 
            
for chapter in chapters:
    print(chapter)
    print(chapters[chapter])

FONDS  DE  CUISINE  
  I 
THE  LEADING  WARM   SAUCES     
  •  '5 
THE   SMALL  COMPOUND   SAUCES  
  24 
COLD  SAUCES  AND  COMPOUND  BUTTERS        
  48 
SAVOURY  JELLIES  OR  ASPICS  
  59 
THE  COURT-BOUILLONS  AND  THE  MARINADES  
  -64 
\J/:  ELEMENTARY  PREPARATIONS  
  70 
THE  VARIOUS  GARNISHES   FOR  SOUPS  
  87 
GARNISHING  PREPARATIONS   FOR  RELEVis  AND   ENTR]£eS  
  92 
U^DING  CULINARY  OPERATIONS  
  97 
HORS-D'CEUVRES      
137 
EGGS  
        164 
SOUPS  
      197 
FISH  
        260 
RELEVilS  AND  ENTRIES  OF  BUTCHER'S  MEAT  
       352 
RELEVES  AND  ENTRIES  OF  POULTRY  AND  GAME    
       473 
ROASTS  AND  SALADS         
       605 
VEGETABLES  AND   FARINACEOUS  PRODUCTS  
       624 
SAVORIES       
       678 
ENTREMETS
       687 
ICES   AND  SHERBETS  
  788 
DRINKS   AND   REFRESHMENTS     
816 
FRUIT-STEWS  AND  JAMS  ,
       820 


These chapters have errors:
FONDS  DE  CUISINE  
THE  LEADING  WARM   SAUCES     
THE  COURT-BOUILLONS  AND  THE  MARINADES  
\J/:  ELEMENTARY  PREPARATIONS  
GARNISHING  PREPARATIONS   FOR  RELEVis  AND   ENTR]£eS  
U^DING  CULINARY  OPERATIONS  
HORS-D'CEUVRES      
RELEVilS  AND  ENTRIES  OF  BUTCHER'S  MEAT  
RELEVES  AND  ENTRIES  OF  POULTRY  AND  GAME    

In [31]:
chapters['FONDS  DE  CUISINE  '] = 1
chapters['THE  LEADING  WARM   SAUCES     '] = 15
chapters['THE  COURT-BOUILLONS  AND  THE  MARINADES  '] = 64
correct_titles = [
    'THE  ELEMENTARY  PREPARATIONS  ',
    'GARNISHING  PREPARATIONS   FOR  RELEVES  AND   ENTREES  ',
    'LEADING  CULINARY  OPERATIONS  ',
    "HORS-D'OEUVRES      ",
    "RELEVES  AND  ENTREES  OF  BUTCHER'S  MEAT  ",
    "RELEVES  AND  ENTREES  OF  POULTRY  AND  GAME    "  
]
incorrect_titles = [
    '\J/:  ELEMENTARY  PREPARATIONS  ',
    'GARNISHING  PREPARATIONS   FOR  RELEVis  AND   ENTR]£eS  ',
    'U^DING  CULINARY  OPERATIONS  ',
    "HORS-D'CEUVRES      ",
    "RELEVilS  AND  ENTRIES  OF  BUTCHER'S  MEAT  ",
    "RELEVES  AND  ENTRIES  OF  POULTRY  AND  GAME    " 
]
for i in range(len(correct_titles)):
    chapters[correct_titles[i]] = chapters[incorrect_titles[i]]
    del chapters[incorrect_titles[i]]

I can disregard the first 3 lines, and there are some minor issues with the text (e.g. "U^DING" instead of "LEADING"). Looking at the PDF, it seems there are some text marks on the original pages of the book that underwent OCR for the PDF. Since there are only a few mistakes, I'll fix these manually just to move forward. If this problem becomes too hindering I'll find a better method.

In [32]:
for chapter in chapters:
    print(chapter)
    print(chapters[chapter])

FONDS  DE  CUISINE  
1
THE  LEADING  WARM   SAUCES     
15
THE   SMALL  COMPOUND   SAUCES  
  24 
COLD  SAUCES  AND  COMPOUND  BUTTERS        
  48 
SAVOURY  JELLIES  OR  ASPICS  
  59 
THE  COURT-BOUILLONS  AND  THE  MARINADES  
64
THE  VARIOUS  GARNISHES   FOR  SOUPS  
  87 
EGGS  
        164 
SOUPS  
      197 
FISH  
        260 
ROASTS  AND  SALADS         
       605 
VEGETABLES  AND   FARINACEOUS  PRODUCTS  
       624 
SAVORIES       
       678 
ENTREMETS
       687 
ICES   AND  SHERBETS  
  788 
DRINKS   AND   REFRESHMENTS     
816 
FRUIT-STEWS  AND  JAMS  ,
       820 
THE  ELEMENTARY  PREPARATIONS  
  70 
GARNISHING  PREPARATIONS   FOR  RELEVES  AND   ENTREES  
  92 
LEADING  CULINARY  OPERATIONS  
  97 
HORS-D'OEUVRES      
137 
RELEVES  AND  ENTREES  OF  BUTCHER'S  MEAT  
       352 
RELEVES  AND  ENTREES  OF  POULTRY  AND  GAME    
       473 
